# Admission Model 

In [29]:
import pandas as pd
df = pd.read_csv('/Users/shubhamgandhi/Downloads/case_study_cleaned_data.csv')

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [30]:
df.shape

(92338, 17)

In [31]:
df.columns

Index(['Student ID', 'Student Name', 'Entry Date', 'Last Activity Date',
       'Latest Response', 'Department', 'Area of Study', 'No Email sent',
       'No SMS sent', 'Total Calls', 'State', 'City', 'Last exam taken',
       'Fee Range ', 'Admission Status', 'OTP Verification', 'Source '],
      dtype='object')

In [32]:
df.head()

,Student ID,Student Name,Entry Date,Last Activity Date,Latest Response,Department,Area of Study,No Email sent,No SMS sent,Total Calls,State,City,Last exam taken,Fee Range,Admission Status,OTP Verification,Source
0,1,Manikanta mk,04-04-2020 16:57,11-05-2020 17:26,General Enquiry,"Mathematics,",Sciences and Mathematics,0,2,2,Karnataka,Bangalore,NaN,NaN,Done,Yes,Online portal
1,2,SAMEEKSHA PAI,14-04-2020 15:09,11-05-2020 17:18,Non Relevent,"Civil Engineering,",Engineering,0,2,5,Karnataka,Bangalore,NaN,NaN,Done,-,Digital Ad
2,3,Mariyamol stephen,28-02-2020 23:03,11-05-2020 17:20,Not Interested,"Nursing,",Nursing,0,2,2,Karnataka,Bangalore,NaN,NaN,Done,-,Online portal
3,4,Maheswarareddy,23-03-2020 21:46,11-05-2020 17:30,Call Disconnected,"Journalism,",Journalism,0,2,2,Karnataka,Bangalore,NaN,NaN,Done,-,Online portal
4,5,amarnath,05-05-2020 18:08,11-05-2020 17:13,Non Relevent,"Computer Application,",Computer Application,0,2,2,Karnataka,Bangalore,NaN,NaN,Done,-,Online portal


## Convert columns to lower case and remove spaces

In [33]:
print('columns names before processing', df.columns)
df.columns = ['_'.join(i.strip().lower().split()) for i in df.columns]
print("="*80)
print('columns names after processing', df.columns)

columns names before processing Index(['Student ID', 'Student Name', 'Entry Date', 'Last Activity Date',
       'Latest Response', 'Department', 'Area of Study', 'No Email sent',
       'No SMS sent', 'Total Calls', 'State', 'City', 'Last exam taken',
       'Fee Range ', 'Admission Status', 'OTP Verification', 'Source '],
      dtype='object')
columns names after processing Index(['student_id', 'student_name', 'entry_date', 'last_activity_date',
       'latest_response', 'department', 'area_of_study', 'no_email_sent',
       'no_sms_sent', 'total_calls', 'state', 'city', 'last_exam_taken',
       'fee_range', 'admission_status', 'otp_verification', 'source'],
      dtype='object')


## Null values

In [34]:
df.isnull().sum()

student_id                0
student_name              0
entry_date                0
last_activity_date        0
latest_response        3801
department                0
area_of_study             0
no_email_sent             0
no_sms_sent               0
total_calls               0
state                   129
city                  70976
last_exam_taken       64657
fee_range             12589
admission_status          0
otp_verification          0
source                    0
dtype: int64

In [36]:
drop_cols = ['city', 'last_exam_taken']
na_cols = ['latest_response', 'state', 'fee_range']
print('df size before dropping high NA value columns',df.shape)
df.drop(drop_cols, inplace=True, axis=1)
print('df size after dropping high NA value columns',df.shape)
print('df size without any null values', df.dropna().shape)

df size before dropping high NA value columns (92338, 17)
df size after dropping high NA value columns (92338, 15)
df size without any null values (76171, 15)


In [37]:
df.dropna(inplace=True)
df.reset_index(drop=True)
print(df.shape)

(76171, 15)


## Create admission flag

In [43]:
df.loc[df.admission_status=='Done', 'admission_flag'] = 1
df.loc[df.admission_status!='Done', 'admission_flag'] = 0

In [44]:
df.admission_flag.value_counts()

0.0    55865
1.0    20306
Name: admission_flag, dtype: int64

In [45]:
df.admission_status.value_counts()

-       55865
Done    20306
Name: admission_status, dtype: int64

# Feature eng

## create variables

### Month of entry date

In [38]:
df.entry_date.head(5)

13    09-05-2020 15:35
14    08-05-2020 19:39
15    10-03-2020 18:58
17    24-03-2020 20:00
18    11-03-2020 22:25
Name: entry_date, dtype: object

<b><p style="font-size:20px">Convert `entry_date` to date time format</p></b>

In [40]:

df.entry_date = pd.to_datetime(df.entry_date)
df.entry_date.dtype
df.entry_date.head(5)

13   2020-09-05 15:35:00
14   2020-08-05 19:39:00
15   2020-10-03 18:58:00
17   2020-03-24 20:00:00
18   2020-11-03 22:25:00
Name: entry_date, dtype: datetime64[ns]

<b><p style="font-size:20px">Create month column from `entry_date`</p></b>

In [41]:
df.loc[:, 'entry_month'] = df.entry_date.dt.strftime("%b")
df.head(5)

,student_id,student_name,entry_date,last_activity_date,latest_response,department,area_of_study,no_email_sent,no_sms_sent,total_calls,state,fee_range,admission_status,otp_verification,source,entry_month
13,14,Thangaraju,2020-09-05 15:35:00,11-05-2020 17:12,Not Interested,"B.E. Engineering,",Engineering,0,2,2,Karnataka,Fee no barrier,-,-,Online portal,Sep
14,15,Yashwanth ks,2020-08-05 19:39:00,11-05-2020 17:12,Not Interested,"B.E. Engineering,",Engineering,0,2,3,Karnataka,Fee no barrier,-,-,Online portal,Aug
15,16,ANU EP,2020-10-03 18:58:00,11-05-2020 17:25,Not Interested,"Journalism,",Journalism,0,2,2,Karnataka,Fee no barrier,Done,-,Online portal,Oct
17,18,Aleesha Babu,2020-03-24 20:00:00,01-09-2020 11:54,Admit in CU,"Nursing,",Nursing,0,2,3,Karnataka,Fee no barrier,Done,-,Emailer,Mar
18,19,Mjhammed cm,2020-11-03 22:25:00,11-05-2020 17:27,Switch Off,"Computer Science,",Engineering,0,2,11,Karnataka,Fee no barrier,Done,Yes,Online portal,Nov


In [46]:
df.dtypes

student_id                     int64
student_name                  object
entry_date            datetime64[ns]
last_activity_date            object
latest_response               object
department                    object
area_of_study                 object
no_email_sent                  int64
no_sms_sent                    int64
total_calls                    int64
state                         object
fee_range                     object
admission_status              object
otp_verification              object
source                        object
entry_month                   object
admission_flag               float64
dtype: object

<b><p style="font-size:20px">Correct levels of `department`</p></b>

In [58]:
df.department[df.department=='Not Known,'] = 'Not Known'
df.department[df.department=='missing'] = 'Not Known'

<ipython-input-58-c0e923183ec2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.department[df.department=='Not Known,'] = 'Not Known'
<ipython-input-58-c0e923183ec2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.department[df.department=='missing'] = 'Not Known'


<b><p style="font-size:20px">Correct levels of `area_of_study`</p></b>

In [61]:
df.area_of_study[df.area_of_study=='Not Known,'] = 'Not Known'
df.area_of_study[df.area_of_study=='missing'] = 'Not Known'

<ipython-input-61-5474e00644fa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.area_of_study[df.area_of_study=='Not Known,'] = 'Not Known'
<ipython-input-61-5474e00644fa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.area_of_study[df.area_of_study=='missing'] = 'Not Known'


# Split the data

In [116]:
df.drop(['admission_status', 'otp_verification'], axis=1, inplace=True)

In [117]:
numeric_cols = ['no_email_sent', 'no_sms_sent', 'total_calls']
categorical_cols = ['entry_month', 'source', 'fee_range', 'state', 'area_of_study', 'department']
target_col = 'admission_flag'
non_target_cols = [i for i in df.columns if i!=target_col]

In [77]:
i = 4
cat_col = categorical_cols[i]
df[cat_col].value_counts()


Business/Management         30069
Engineering                 18400
Sciences and Mathematics    10235
Computer Application         6477
Journalism                   5290
Nursing                      3803
Hospitality                  1101
Fashion Design                774
Not Known                      22
Name: area_of_study, dtype: int64

In [118]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[non_target_cols], df[target_col], 
                                                    test_size=0.30, random_state=42, stratify=df[target_col])

In [119]:
print('rows in train set: {0: >4}'.format( X_train.shape[0]))
print('rows in test set : {0: >4}'.format( X_test.shape[0]))

rows in train set: 53319
rows in test set : 22852


# Encode Categorical Data

In [120]:
import category_encoders as ce
encoder = ce.TargetEncoder(cols=categorical_cols)

encoder.fit(X_train, y_train)
X_train = encoder.transform(X_train)
X_test  = encoder.transform(X_test)


In [121]:
X_train.head()

,student_id,student_name,entry_date,last_activity_date,latest_response,department,area_of_study,no_email_sent,no_sms_sent,total_calls,state,fee_range,source,entry_month
72241,72242,Shreedhar G H,2020-08-25 21:50:00,25-08-2020 21:50,Admit in Other College,0.251021,0.251021,1,1,3,0.189225,0.189465,0.25678,0.236273
87032,87033,Prasatwika N,2020-09-30 08:32:00,30-09-2020 08:32,Not Interested,0.355687,0.258448,1,1,3,0.035115,0.189465,0.25678,0.174253
53344,53345,Bhavana,2020-05-09 13:59:00,05-09-2020 13:59,Call Disconnected,0.321534,0.258448,0,0,18,0.496403,0.189465,0.25678,0.424291
50596,50597,Aachi,2020-02-08 16:18:00,02-08-2020 16:28,Call Disconnected,0.244726,0.244726,1,1,5,0.496403,0.586385,0.25678,0.180548
57954,57955,Hemanth K H,2020-08-25 21:58:00,25-08-2020 21:58,Admit in Other College,0.302257,0.226146,1,1,3,0.120133,0.189465,0.25678,0.236273


# Standard Scale

In [138]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(X_train[numeric_cols])
X_train.loc[:, numeric_cols] = ss.transform(X_train[numeric_cols])
X_test.loc[:, numeric_cols] = ss.transform(X_test[numeric_cols])

# Models

In [139]:
model_cols = ['no_email_sent', 'no_sms_sent', 'total_calls'] + ['entry_month', 'source', 'fee_range', 'state', 'area_of_study', 'department']


## Linear Model

In [140]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=42)
lr.fit(X_train[model_cols], y_train)

LogisticRegression(random_state=42)

In [141]:
from sklearn.metrics import accuracy_score
tr_predicts = lr.predict(X_train[model_cols])
te_predicts = lr.predict( X_test[model_cols])

In [142]:
print(accuracy_score(y_train, tr_predicts))
print(accuracy_score(y_test,  te_predicts))

0.750539207411992
0.746542972168738


In [143]:
ft_imp = pd.DataFrame([model_cols, *lr.coef_]).transpose()
ft_imp.columns = ['feature', 'coefficient']
ft_imp.sort_values('coefficient', inplace=True)
ft_imp.reset_index(drop=True, inplace=True)
ft_imp.head(10)

,feature,coefficient
0,no_email_sent,-1.21675
1,area_of_study,-0.493144
2,total_calls,-0.035182
3,no_sms_sent,1.45377
4,entry_month,2.58429
5,fee_range,3.01604
6,department,4.44647
7,state,4.6378
8,source,4.8361


### Get p values

In [157]:
import numpy as np
import statsmodels.api as sm
x_tr = sm.add_constant(X_train[model_cols])
x_te = sm.add_constant(X_test[model_cols])

model = sm.Logit(y_train, x_tr)
result = model.fit(method='newton')

Optimization terminated successfully.
         Current function value: 0.491215
         Iterations 6


In [158]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:         admission_flag   No. Observations:                53319
Model:                          Logit   Df Residuals:                    53309
Method:                           MLE   Df Model:                            9
Date:                Mon, 17 May 2021   Pseudo R-squ.:                  0.1528
Time:                        14:28:03   Log-Likelihood:                -26191.
converged:                       True   LL-Null:                       -30916.
Covariance Type:            nonrobust   LLR p-value:                     0.000
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -6.4064      0.141    -45.391      0.000      -6.683      -6.130
no_email_sent    -1.2199      0.063    -19.474      0.000      -1.343      -1.097
no_sms_sent       1.4577      0.071     20.575      0.000       1.319       1.597
total_calls      -0.0353      0.011     -3.178      0.001      -0.057      -0.014
entry_month       2.6113      0.129     20.305      0.000       2.359       2.863
source            5.1670      0.256     20.177      0.000       4.665       5.669
fee_range         3.0538      0.118     25.928      0.000       2.823       3.285
state             4.6644      0.080     58.082      0.000       4.507       4.822
area_of_study    -0.8467      0.469     -1.807      0.071      -1.765       0.072
department        4.7537      0.250     19.022      0.000       4.264       5.244
=================================================================================
"""

## CatboostBoost

### fit model

In [177]:
from catboost import CatBoostClassifier
cbc = CatBoostClassifier(random_seed=42, metric_period=200)

In [178]:
cbc.fit(X_train[model_cols], y_train)

Learning rate set to 0.056275
0:	learn: 0.6656044	total: 13.3ms	remaining: 13.3s
200:	learn: 0.4042671	total: 1.54s	remaining: 6.14s
400:	learn: 0.3944790	total: 3.43s	remaining: 5.13s
600:	learn: 0.3887798	total: 6.08s	remaining: 4.04s
800:	learn: 0.3843239	total: 9.47s	remaining: 2.35s
999:	learn: 0.3805846	total: 11.4s	remaining: 0us


In [179]:
cbc_tr_predicts = cbc.predict(X_train[model_cols])
cbc_te_predicts = cbc.predict(X_test[model_cols])

### Model evaluation

In [180]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_train, cbc_tr_predicts))
print(accuracy_score(y_test, cbc_te_predicts))

0.8189575948536169
0.8054437248380886


### feature importance

In [185]:
cbc_ft_imp = pd.DataFrame([model_cols, cbc.get_feature_importance()]).transpose()
cbc_ft_imp.columns = ['feature', 'ft_imp']
cbc_ft_imp.sort_values('ft_imp', inplace=True, ascending=False)
cbc_ft_imp.reset_index(drop=True, inplace=True)
cbc_ft_imp.head()

,feature,ft_imp
0,state,33.7991
1,fee_range,26.1136
2,department,11.2857
3,entry_month,8.77979
4,no_email_sent,7.77059
